In [2]:
import sys, os
os.chdir(os.path.dirname(os.path.abspath("./")))
print(os.getcwd())

# SCRIPT_DIR = os.path.abspath(".")
# print(SCRIPT_DIR)
# sys.path.append(SCRIPT_DIR)
# sys.path


/home/konstfed/Documents/Study/PMLDL/PMLDL-TextDetoxification


In [3]:

import yaml
from addict import Dict

from src.models import build_model
from src.inference import ToxicClassificationPipeline, BertPipeline
from src import preprocessing

sys.modules['preprocessing'] = preprocessing

/home/konstfed/Documents/Study/PMLDL/PMLDL-TextDetoxification/toxic-venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/konstfed/Documents/Study/PMLDL/PMLDL-TextDetoxification/toxic-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open("configs/sentence_toxic_cls/hard_labels.yaml", "r") as f:
    hard_config = Dict(yaml.safe_load(f))

with open("configs/sentence_toxic_cls/soft_labels.yaml", "r") as f:
    soft_config = Dict(yaml.safe_load(f))

with open("configs/sentence_toxic_cls/distilbert_inference.yaml", "r") as f:
    bert_config = Dict(yaml.safe_load(f))


soft_cls = ToxicClassificationPipeline(soft_config) # BCEloss 0.45 on test
hard_cls = ToxicClassificationPipeline(hard_config) # BCEloss 0.46 on test
bert_cls = BertPipeline(bert_config) # BCEloss 0.05 on test

In [16]:
texts2analyze = [
    "You are moron",
    "I'm so lonely and miserable. Life is so hard.",
    "Ignore the dick",
    "Whoever wrote this is a waste of space",
    "I love cats",
    "If you have not time to grade our works, why you think that we have enough time to write 7 pages of report with full-time work?",
    "Also no bonus points for late grading. Half of works are not graded. The worst course at this moment",
]

In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [17]:
for i, text in enumerate(texts2analyze,start=1):
    soft_res = soft_cls.forward(text)
    hard_res = hard_cls.forward(text)
    bert_res = bert_cls.forward(text)
    print(text)
    print(f"soft label: {soft_res}, hard_label: {hard_res}, bert label: {bert_res}")
    print()

You are moron
soft label: tensor([0.5065]), hard_label: tensor([0.4943]), bert label: tensor([[0.7359]])

I'm so lonely and miserable. Life is so hard.
soft label: tensor([0.5045]), hard_label: tensor([0.4944]), bert label: tensor([[0.5495]])

Ignore the dick
soft label: tensor([0.5043]), hard_label: tensor([0.4969]), bert label: tensor([[0.7343]])

Whoever wrote this is a waste of space
soft label: tensor([0.5053]), hard_label: tensor([0.4941]), bert label: tensor([[0.5132]])

I love cats
soft label: tensor([0.5060]), hard_label: tensor([0.4938]), bert label: tensor([[0.5011]])

If you have not time to grade our works, why you think that we have enough time to write 7 pages of report with full-time work?
soft label: tensor([0.5051]), hard_label: tensor([0.4936]), bert label: tensor([[0.5027]])

Also no bonus points for late grading. Half of works are not graded. The worst course at this moment
soft label: tensor([0.5056]), hard_label: tensor([0.4946]), bert label: tensor([[0.5116]])

